In [2]:
import numpy as np
TRAIN_FILE = 'fashion_mnist/train.csv'
TEST_FILE = 'fashion_mnist/test.csv'
VAL_FILE = 'fashion_mnist/val.csv'

In [201]:
data = np.genfromtxt(TRAIN_FILE,delimiter=',')

In [202]:
data.shape

(22500, 785)

In [203]:
classes = [0,1] # Binary classification 
newdata = np.asarray([r for r in data if r[-1] in classes])

In [204]:
# change classes to {-1,1}
# change values to [0,1] from [0,255]
X = newdata[:,0:784] / 255.0
Y = 2 * np.asarray(newdata[:,-1]) -1 
Y = np.asarray([Y]).astype(float)

In [205]:
# linear kernel
Z = Y.T * X

In [206]:
C = 1.0

In [207]:
## M x M array Linear kernel
P = np.matmul(Z, Z.T)

In [208]:
# column vector with all -1
Q = -1 * np.ones((len(P),1)).astype(float)

In [209]:
G = -1 * np.eye(len(P)).astype(float)
h = np.zeros((len(P),1)).astype(float)

In [210]:
Gp = np.eye(len(P)).astype(float)
hp = C * np.ones((len(P),1)).astype(float)

In [211]:
Gt = np.vstack((G,Gp))
ht = np.vstack((h,hp))

In [212]:
G = Gt
h = ht

In [213]:
A = Y
B = [[0]]

In [214]:
import cvxopt as cv


In [215]:
A_ = cv.matrix(A)
P_ = cv.matrix(P)
Q_ = cv.matrix(Q)
G_ = cv.matrix(G)
h_ = cv.matrix(h)
b_ = cv.matrix(B,tc='d')

In [216]:
sol = cv.solvers.qp(P_,Q_,G_,h_,A_,b_)

     pcost       dcost       gap    pres   dres
 0: -3.2916e+02 -9.3878e+03  5e+04  3e+00  3e-12
 1: -2.0018e+02 -5.2391e+03  1e+04  4e-01  2e-12
 2: -9.3627e+01 -1.6503e+03  3e+03  8e-02  1e-12
 3: -5.1870e+01 -7.1316e+02  1e+03  3e-02  8e-13
 4: -2.9393e+01 -4.7263e+02  7e+02  2e-02  5e-13
 5: -1.8616e+01 -2.5482e+02  4e+02  7e-03  4e-13
 6: -1.5942e+01 -8.1551e+01  9e+01  2e-03  4e-13
 7: -1.7889e+01 -4.1759e+01  3e+01  4e-04  4e-13
 8: -1.9475e+01 -3.1857e+01  1e+01  8e-05  4e-13
 9: -1.9419e+01 -2.8941e+01  1e+01  3e-16  4e-13
10: -2.1921e+01 -2.4878e+01  3e+00  9e-16  4e-13
11: -2.2590e+01 -2.3752e+01  1e+00  2e-16  4e-13
12: -2.3023e+01 -2.3203e+01  2e-01  4e-15  4e-13
13: -2.3099e+01 -2.3108e+01  1e-02  2e-15  4e-13
14: -2.3103e+01 -2.3103e+01  2e-04  5e-15  4e-13
15: -2.3103e+01 -2.3103e+01  2e-06  9e-16  4e-13
Optimal solution found.


In [217]:
alph = np.array(sol['x'])

In [218]:
fw = alph * Y.T

In [219]:
w = np.matmul(fw.T,X)

In [220]:
R = np.matmul(w,X.T)

In [221]:
maxv = 0.0
minv = 100000000.0
for i in range(0,len(R[0])):
    if Y[0][i] == 1:
        if(R[0][i] < minv):
            minv = R[0][i]
    else:
        if(R[0][i]>maxv):
            maxv = R[0][i]
b = -(maxv+minv)/2.0

In [222]:
# calculating support vectors
sX = Y * (np.matmul(w,X.T) + b)
vectors = np.where((sX-1) < 0.0001)[1]

In [105]:
vectors

(191,)

In [223]:
tdata = np.genfromtxt(TEST_FILE,delimiter=',')

In [224]:
tY = tdata[:,-1]
tX = tdata[:,0:784]/255.0


In [225]:
tx = []
ty = []
for i in range(0,len(tX)):
    if tY[i] not in classes:
        continue;
    else:
        ty.append(tY[i])
        tx.append(tX[i])
tX = np.asarray(tx)
tY = np.asarray(ty)

In [226]:

tR = np.matmul(w,tX.T) + b

In [227]:
tot = 0
corr = 0
for i in range(0,len(tR[0])):
    if(tY[i] not in classes):
        continue;
    tot += 1;
    if((tR[0][i]>0 and tY[i]==1) or (tR[0][i]<0 and tY[i]==0)):
        corr +=1;

In [228]:
print(float(corr)/float(tot))

0.982


In [106]:
import math

In [127]:
# Gaussian Kernel
def gaussian(y,v):
#     x = np.asarray([Xi - Xj])
    return math.exp(-y * v)

In [126]:
a = np.ones((2,3))
b = np.ones((2,3))
np.sum(a*b,axis=1)

array([3., 3.])

In [129]:
y = 0.05
P = np.eye(len(X)).astype(float)
for i in range(0,len(X)):
    t = X - X[i]
    P[i] = np.sum(t*t,axis=1)

In [135]:
g = np.vectorize(gaussian)
pp = g(y,P)

In [143]:
# P matrix
pp = np.matmul(Y.T,Y) * pp

In [148]:
A_ = cv.matrix(A)
P_ = cv.matrix(pp)
Q_ = cv.matrix(Q)
G_ = cv.matrix(G)
h_ = cv.matrix(h)
b_ = cv.matrix(B,tc='d')

In [149]:
sol = cv.solvers.qp(P_,Q_,G_,h_,A_,b_)

     pcost       dcost       gap    pres   dres
 0: -1.6328e+02 -5.5786e+02  1e+04  9e+01  2e+00
 1: -9.9994e+01 -5.6249e+02  2e+03  1e+01  3e-01
 2: -1.2081e+02 -3.0325e+02  4e+02  1e+00  4e-02
 3: -1.4358e+02 -2.1550e+02  1e+02  4e-01  9e-03
 4: -1.5662e+02 -1.8264e+02  3e+01  5e-02  1e-03
 5: -1.6415e+02 -1.7387e+02  1e+01  1e-02  3e-04
 6: -1.6748e+02 -1.7122e+02  4e+00  2e-15  2e-15
 7: -1.6908e+02 -1.7028e+02  1e+00  1e-14  2e-15
 8: -1.6975e+02 -1.6998e+02  2e-01  1e-14  2e-15
 9: -1.6992e+02 -1.6994e+02  2e-02  3e-15  2e-15
10: -1.6993e+02 -1.6993e+02  9e-04  8e-15  2e-15
11: -1.6993e+02 -1.6993e+02  1e-05  4e-15  2e-15
Optimal solution found.


In [150]:
alph = np.array(sol['x'])

In [152]:
fW = alph * Y.T

In [160]:
a = tX - X[0]
# a.shape


(5000, 784)

In [194]:
# Test
tx = np.ones((len(X),len(tX))).astype(float)
for i in range(0,len(X)):
    tt = tX - X[i]
    tx[i] = np.sum(tt*tt,axis = 1)

In [195]:
tx

array([[144.07315648,  33.50499039,  71.04527489, ...,  40.89783929,
         21.0779085 , 184.35391003],
       [127.96532103,  39.82677432,  65.94495963, ...,  33.71847751,
         28.43092657, 163.42097655],
       [145.8973318 ,  89.98795848,  19.52381392, ...,  65.9649827 ,
         68.02629758, 170.00504421],
       ...,
       [ 81.62497501,  85.12918108,  13.19649366, ...,  67.50928105,
         66.45028835, 109.40432141],
       [106.17619377,  45.25880815,  73.55455594, ...,  50.54618993,
         23.69302576, 138.10818916],
       [181.3127105 , 129.90003845,  62.71672434, ..., 113.07257209,
        107.64393695, 190.71124952]])

In [186]:
Gg = g(y,tx)

In [187]:
Gg

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [196]:
R = np.sum(fw * g(y,tx), axis=0)

In [197]:
corr = 0
for i in range(0,len(R)):
    if((R[i] > 0 and tY[i]==1 )or(R[i]<0 and tY[i] ==0)):
        corr +=1

In [200]:
corr/float(len(R))

0.892

In [55]:
from sklearn import svm
clf = svm.SVC(kernel='linear')

In [56]:
clf.fit(X,Y[0])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [104]:
clf.support_

array([   3,  106,  127,  171,  232,  233,  234,  301,  367,  436,  494,
        500,  616,  775,  800,  844,  878,  885,  970,  980,  986, 1005,
       1107, 1114, 1134, 1153, 1239, 1333, 1381, 1394, 1437, 1489, 1511,
       1563, 1619, 1639, 1658, 1683, 1684, 1716, 1729, 1798, 1819, 1845,
       1895, 1905, 1931, 1991, 1993, 2011, 2172, 2217, 2271, 2300, 2315,
       2325, 2344, 2355, 2420, 2560, 2591, 2610, 2658, 2691, 2793, 2813,
       2851, 2859, 2873, 2886, 3023, 3048, 3052, 3107, 3157, 3164, 3184,
       3257, 3263, 3310, 3340, 3402, 3545, 3551, 3585, 3593, 3611, 3626,
       3634, 3645, 3654, 3711, 3734, 3753, 3759, 3771, 3797, 3822, 3939,
       3979, 3995, 4061, 4171, 4243, 4271, 4290, 4304, 4320, 4413, 4445,
       4454,   18,   27,   29,   63,  110,  137,  255,  306,  357,  387,
        448,  489,  519,  520,  622,  637,  985, 1162, 1173, 1286, 1317,
       1351, 1356, 1391, 1426, 1431, 1438, 1448, 1449, 1581, 1647, 1691,
       1710, 1711, 1825, 1978, 2001, 2133, 2205, 22